In [1]:
import json 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
from tensorflow.keras.layers import LSTM

In [3]:
import pickle

In [4]:
with open("../Machine Learning/intents.json") as datafile:
    data = json.load(datafile)
df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
75,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [7]:
df.isnull().sum()

tag          0
patterns     0
responses    0
dtype: int64

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
    
num_classes = len(labels)

In [7]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences,truncating='post', maxlen = max_len)

In [8]:
from tensorflow.keras.layers import LSTM, Dropout



model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(LSTM(64))  # You can adjust the number of units in the LSTM layer as needed
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(num_classes, activation='softmax'))



In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
epochs = 510
result = model.fit(padded_sequences, np.array(training_labels), epochs = epochs)

Epoch 1/510
10/10 [==============================] - 2s 7ms/step - loss: 4.3768 - accuracy: 0.0345
Epoch 2/510
10/10 [==============================] - 0s 7ms/step - loss: 4.3511 - accuracy: 0.0502
Epoch 3/510
10/10 [==============================] - 0s 7ms/step - loss: 4.2579 - accuracy: 0.0502
Epoch 4/510
10/10 [==============================] - 0s 8ms/step - loss: 4.1675 - accuracy: 0.0502
Epoch 5/510
10/10 [==============================] - 0s 9ms/step - loss: 4.0962 - accuracy: 0.0470
Epoch 6/510
10/10 [==============================] - 0s 8ms/step - loss: 4.1070 - accuracy: 0.0502
Epoch 7/510
10/10 [==============================] - 0s 9ms/step - loss: 4.0728 - accuracy: 0.0439
Epoch 8/510
10/10 [==============================] - 0s 9ms/step - loss: 4.0161 - accuracy: 0.0470
Epoch 9/510
10/10 [==============================] - 0s 9ms/step - loss: 3.9746 - accuracy: 0.0439
Epoch 10/510
10/10 [==============================] - 0s 7ms/step - loss: 3.9310 - accuracy: 0.0690
Epoch 11/

10/10 [==============================] - 0s 7ms/step - loss: 1.2876 - accuracy: 0.7210
Epoch 84/510
10/10 [==============================] - 0s 7ms/step - loss: 1.2462 - accuracy: 0.7429
Epoch 85/510
10/10 [==============================] - 0s 7ms/step - loss: 1.2365 - accuracy: 0.7210
Epoch 86/510
10/10 [==============================] - 0s 7ms/step - loss: 1.2221 - accuracy: 0.6928
Epoch 87/510
10/10 [==============================] - 0s 8ms/step - loss: 1.2337 - accuracy: 0.7461
Epoch 88/510
10/10 [==============================] - 0s 8ms/step - loss: 1.1287 - accuracy: 0.7649
Epoch 89/510
10/10 [==============================] - 0s 8ms/step - loss: 1.1647 - accuracy: 0.7555
Epoch 90/510
10/10 [==============================] - 0s 7ms/step - loss: 1.1355 - accuracy: 0.7555
Epoch 91/510
10/10 [==============================] - 0s 7ms/step - loss: 1.1360 - accuracy: 0.7429
Epoch 92/510
10/10 [==============================] - 0s 8ms/step - loss: 1.1168 - accuracy: 0.7743
Epoch 93/510


10/10 [==============================] - 0s 7ms/step - loss: 0.1136 - accuracy: 0.9875
Epoch 246/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1510 - accuracy: 0.9781
Epoch 247/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1178 - accuracy: 0.9875
Epoch 248/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1206 - accuracy: 0.9906
Epoch 249/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1196 - accuracy: 0.9812
Epoch 250/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1067 - accuracy: 0.9875
Epoch 251/510
10/10 [==============================] - 0s 7ms/step - loss: 0.1362 - accuracy: 0.9749
Epoch 252/510
10/10 [==============================] - 0s 6ms/step - loss: 0.1058 - accuracy: 0.9875
Epoch 253/510
10/10 [==============================] - 0s 6ms/step - loss: 0.1236 - accuracy: 0.9843
Epoch 254/510
10/10 [==============================] - 0s 6ms/step - loss: 0.1079 - accuracy: 0.9875
Epoc

10/10 [==============================] - 0s 7ms/step - loss: 0.0473 - accuracy: 0.9906
Epoch 408/510
10/10 [==============================] - 0s 8ms/step - loss: 0.0447 - accuracy: 0.9937
Epoch 409/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0533 - accuracy: 0.9843
Epoch 410/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0692 - accuracy: 0.9875
Epoch 411/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0462 - accuracy: 0.9937
Epoch 412/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0456 - accuracy: 0.9875
Epoch 413/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0557 - accuracy: 0.9843
Epoch 414/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0503 - accuracy: 0.9843
Epoch 415/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0404 - accuracy: 0.9875
Epoch 416/510
10/10 [==============================] - 0s 7ms/step - loss: 0.0487 - accuracy: 0.9875
Epoc

In [11]:
# to save the train model
model.save("chat_model")

import pickle

#to save the fitted tokenizer
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

#to save the fitted label encoder
with open('label_encoder.pickle','wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol= pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


In [12]:
import colorama
colorama.init()
from colorama import Fore,Style,Back

In [ ]:
def chat():
    #load trained model
    model = keras.models.load_model('chat_model')
    
    #load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
        
    #load label encoder object
    with open('label_encoder.pickle','rb') as enc:
        lbl_encoder = pickle.load(enc)
        
    #parameters
    max_len = 20 
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User:" + Style.RESET_ALL, end ="")
        inp = input()
        if inp.lower() == "quit":
            break
            
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), truncating = 'post', maxlen = max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        
        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(i['responses']) )
                
                # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses)) 
                
print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: